In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from utils import aggregate_by_ticker, get_last_closing_price, get_full_price_history

## Import

In [ ]:
io_path = Path('..','data','in')

In [ ]:
df_storico = pd.read_excel(
    io_path / Path('pac.xlsx'),
    sheet_name='Storico',
    dtype={
        'Borsa': str,
        'Ticker': str,
        'Quote': int,
        'Prezzo (€)': float,
        'Commissioni': float,
    }
).rename(
    columns={
        'Borsa': 'exchange',
        'Ticker': 'ticker',
        'Data Operazione': 'transaction_date',
        'Quote': 'shares',
        'Prezzo (€)': 'price',
        'Commissioni (€)': 'fees',
    }
)

In [ ]:
df_anagrafica = pd.read_excel(
    io_path / Path('pac.xlsx'),
    sheet_name='Anagrafica Titoli',
    dtype=str
).rename(
    columns={
        'Ticker': 'ticker',
        'Nome ETF': 'name',
        'Tipologia': 'asset_class',
        'Macro Tipologia': 'macro_asset_class',
    }
)

In [ ]:
df_pf = aggregate_by_ticker(df_storico, in_pf_only=True)

## Ultima chiusura

In [ ]:
ticker_list = df_pf['ticker_yf'].to_list()

df_last_closing = get_last_closing_price(ticker_list=ticker_list)

## PMC *vs* prezzo attuale

In [ ]:
df_j = df_pf[['ticker_yf','dca','shares']].merge(
    df_last_closing[['ticker_yf','price']],
    how='left',
    on='ticker_yf'
)

df_j['gain'] = np.where(
    df_j['price'].gt(df_j['dca']),
    True,
    False,
)

## PnL

In [ ]:
expense = (
    df_j['shares'] * df_j['dca']
).sum()

In [ ]:
fees = df_storico['fees'].sum().round(2)

In [ ]:
pf_actual_value = (
    df_j['shares'] * df_j['price']
).sum()

In [ ]:
(pf_actual_value - expense).round(2)

In [ ]:
np.round(
    100 * (pf_actual_value - expense) / expense,
    1
)

In [ ]:
(pf_actual_value - expense - fees).round(2)

In [ ]:
np.round(
    100 * (pf_actual_value - expense - fees) / expense,
    1
)

## Pivot per tipologia

In [ ]:
df_j['ticker'] = df_j['ticker_yf'].str.split('.').str[0]
df_j['position_value'] = df_j['shares'] * df_j['price']

In [ ]:
df_pivot = df_j.merge(
    df_anagrafica,
    how='left',
    on='ticker'
).groupby(
    [
        'macro_asset_class',
        'asset_class',
        'ticker',
        'name',
    ]
)['position_value'].sum().reset_index()

In [ ]:
df_pivot['weight_pf'] = (
    100 * df_pivot['position_value'].div(pf_actual_value)
).astype(float).round(1)

In [ ]:
pd.pivot_table(
    df_pivot,
    values=['weight_pf'],
    index=['macro_asset_class', 'asset_class'],
    aggfunc='sum',
    margins=True,
    margins_name='Total',
)

## Full History

In [ ]:
df_full_history = get_full_price_history(ticker_list)

In [ ]:
df_full_history_concat = pd.concat(
    [df_full_history[t_] for t_ in ticker_list],
    axis=1,
)

In [ ]:
# First not-null row
first_idx = df_full_history_concat.apply(
    pd.Series.first_valid_index
).max()

df = df_full_history_concat.loc[first_idx:]

print(f'Starting from {str(first_idx)[:10]} ({df.shape[0]} days, {round(df.shape[0]/252, 1)} yrs)')

## Grafichetti

[tipo](https://plotly.com/python/horizontal-bar-charts/)

In [ ]:
df_pivot

In [ ]:
fig = px.sunburst(
    df_pivot,
    path=['macro_asset_class','asset_class','ticker'],
    values='position_value',
)

fig.show()

## Correlation

In [ ]:
def color_df(val: float) -> str:
    if val <= 0.3:
        color = 'darkblue'
    elif (val > 0.3 and val <= 0.7):
        color = 'darkorange'
    elif (val > 0.7 and val < 1.0):
        color = 'darkred'
    elif val == 1.0:
        color = 'white'
    return 'color: %s' % color

In [ ]:
df_corr = df.corr()

df_corr.style.applymap(color_df)

In [ ]:
mask = np.tril(
    np.ones_like(df_corr, dtype=bool)
)

fig = go.Figure(go.Heatmap(
    z=df_corr.mask(mask),
    x=df_corr.columns,
    y=df_corr.columns,
    colorscale=px.colors.diverging.RdBu,
    reversescale=True,
    zmin=-1,
    zmax=1
))

fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)

fig.show()

## PyPortfolioOpt

[risk-free rate](https://www.ecb.europa.eu/stats/financial_markets_and_interest_rates/euro_short-term_rate/html/index.en.html) area Euro

[Fred](https://fred.stlouisfed.org/series/ECBESTRVOLWGTTRMDMNRT)

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns, plotting

# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# Risk-free rate
risk_free_rate = 0.0315

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
max_sharpe_weights = ef.max_sharpe(risk_free_rate=risk_free_rate)
ef.portfolio_performance(verbose=True, risk_free_rate=risk_free_rate);

In [ ]:
for it_ in max_sharpe_weights.items():
    print(it_)

In [ ]:
ef_plt = EfficientFrontier(mu, S)

fig, ax = plt.subplots()
plotting.plot_efficient_frontier(
    ef_plt,
    ax=ax,
    show_assets=True,
)
plt.show()

In [ ]:
ef_plt = EfficientFrontier(mu, S)

fig, ax = plt.subplots()
ef_max_sharpe = ef_plt.deepcopy()
plotting.plot_efficient_frontier(
    ef_plt,
    ax=ax,
    show_assets=False,
)

# Find the tangency portfolio
ef_max_sharpe.max_sharpe()
ret_tangent, std_tangent, _ = ef_max_sharpe.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Max Sharpe")

# Generate random portfolios
n_samples = 10000
w = np.random.dirichlet(np.ones(ef_plt.n_assets), n_samples)
rets = w.dot(ef_plt.expected_returns)
stds = np.sqrt(np.diag(w @ ef_plt.cov_matrix @ w.T))
sharpes = rets / stds
ax.scatter(stds, rets, marker=".", c=sharpes, cmap="viridis_r")

# Output
ax.set_title("Efficient Frontier with random portfolios")
ax.set_xlim((0.0, 1.0))
ax.set_ylim((0.0, 0.2))
ax.legend()
plt.tight_layout()
plt.show()

## Efficient Frontier

In [ ]:
df_returns = df.pct_change()[1:]

In [ ]:
# Annualized returns (cumulative appreciation)
r = (
    (1 + df_returns).prod()
)**(
    252 / df_returns.shape[0]
) - 1

In [ ]:
# Covariance matrix
cov = 252 * df_returns.cov()

In [ ]:
e = np.ones(r.shape[0])

In [ ]:
# Investable universe
icov = np.linalg.inv(cov)

h = np.matmul(e, icov)
g = np.matmul(r, icov)

a = np.sum(e * h)
b = np.sum(r * h)
c = np.sum(r * g)
d = a * c - b**2

In [ ]:
# MVP (minimum-variance portfolio)
mvp = h / a
mvp_return = b / a
mvp_risk = 1 / np.sqrt(a)

In [ ]:
# Tangency portfolio (with zero risk-free rate)
tangency = g / b
tangency_return = c / b
tangency_risk = np.sqrt(c) / b

In [ ]:
mvp_return, mvp_risk

## Efficient Frontier again

In [ ]:
def OLD_get_returns_and_covariance(df: pd.DataFrame):
    tickers = df.columns.to_list()
    
    returns = df.pct_change()[1:]
    
    r = ((1 + returns).prod())**(252 / returns.shape[0]) - 1
    
    cov = 252 * returns.cov()
    
    return r, cov

In [ ]:
def get_returns_and_covariance(df: pd.DataFrame):
    tickers = df.columns.to_list()
    
    returns = df.pct_change()[1:]
    
    r = returns.mean()
    cov = returns.cov()
    
    return r, cov

In [ ]:
def portfolio_performance(weights, mean_returns, cov):
    returns = 252 * np.sum(mean_returns * weights)
    std = np.sqrt(
        np.dot(
            weights.T,
            np.dot(
                cov,
                weights)
        )
    ) * np.sqrt(252)
    return round(100 * returns, 2), round(100 * std, 2)

In [ ]:
df.columns.to_list()

In [ ]:
dict(
    zip(df_pivot['ticker'], df_pivot['weight_pf'])
)

In [ ]:
weights = np.array(
    [
        0.546,
        0.027,
        0.010,
        0.005,
        0.144,
        0.029,
        0.089,
        0.049,
        0.037,
        0.024,
        0.040,
    ]
)

mean_returns, cov = get_returns_and_covariance(df)

portfolio_performance(weights=weights, mean_returns=mean_returns, cov=cov)

In [ ]:
import scipy as sc

def negative_SR(weights, mean_returns, cov, risk_free_rate = 0):
    pf_returns, pf_std = portfolio_performance(weights, mean_returns, cov)
    return - (pf_returns - risk_free_rate) / pf_std

def max_SR(mean_returns, cov, risk_free_rate = 0, constraint_set=(0,1)):
    "Minimize the negative SR, by altering the weights of the portfolio"
    n_assets = mean_returns.shape[0]
    args = (mean_returns, cov, risk_free_rate)
    constraints = (
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
    )
    bound = constraint_set
    bounds = tuple(bound for asset in range(n_assets))
    result = sc.optimize.minimize(
        negative_SR,
        np.repeat(1/n_assets, n_assets),
        args=args,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
    )
    return result

In [ ]:
negative_SR(weights, mean_returns, cov)

In [ ]:
max_SR_result = max_SR(mean_returns, cov)
max_SR_val, max_SR_weights = max_SR_result['fun'], max_SR_result['x']

In [ ]:
max_SR_val, max_SR_weights